# Seattle, WA - OpenStreetMap Data Wrangling with MongoDB

This project will wrangle and analyze data from the OpenStreetMap project in the Seattle, WA area.

The data was downloaded from this link: https://mapzen.com/data/metro-extracts/metro/seattle_washington/

The code below imports libraries necessary for this project, the OSM file name and regex expressions that will be used later.

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.cElementTree as ET
import pprint
import re
import codecs
import json
from collections import defaultdict

OSMFILE = "seattle_washington.osm"

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

As part of this project, the street names will be standardized. In this dataset, a few incosistencies with street names and abbreviations were found. For example, there are several streets which include a compass direction in their name (Northeast, Northwest, Southeast, Southwest, etc). Different conventions were found, some streets were abbreviated as N.E. while others just used NE and some included the full word. The mapping below will be used to standardize those names.

In [2]:
mapping = { "St": "Street",
            "St.": "Street",
            "Rd": "Road",
            "Rd.": "Road",
            "Ave": "Avenue",
            "Hwy": "Highway",
            "NE": "Northeast",
            "N.E.": "Northeast",
            "NW": "Northwest",
            "N.W.": "Northwest",
            "S": "South",
            "SE": "Southeast",
            "S.E.": "Southeast",
            "SW": "Southwest",
            "S.W.": "Southwest",
            "W" : "West",
            "E" : "East"
            }

The is_street_name function will verify if a particular XML element is a street name.

In [3]:
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

The update_name function will use the mapping variable above to standardize street names.

In [4]:
def update_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        for key, value in mapping.iteritems():
            if street_type == key:
                name = name.replace(key,value)

    return name

The shape_element function receives an XML element and converts it to JSON format to later be inserted in the database. 

In [5]:
def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        lat = str(element.get("lat"))
        lon = str(element.get("lon"))
        try:
            node["pos"] = [float(lat),float(lon)]
        except:
            pass
        created = {}
        created["changeset"] = element.get("changeset")
        created["user"] = element.get("user")
        created["version"] = element.get("version")
        created["uid"] = element.get("uid")
        created["timestamp"] = element.get("timestamp")
        node["created"] = created
        node["visible"] = element.get("visible")
        node["type"] = element.tag
        node["id"] = element.get("id")
        
        address = {}
        for subelement in element.iter("tag"):
            k_element = subelement.get("k")
            v_element = subelement.get("v")
            if not problemchars.match(k_element):
                if k_element.startswith("addr:"):
                    if is_street_name(subelement):
                        v_element = update_name(v_element,mapping)
                    k_elements = k_element.split(":")
                    if(len(k_elements) < 3):
                        address[k_elements[1]] = v_element
                else:
                    node[k_element] = v_element
        if(bool(address)):
            node["address"] = address
            
        if element.tag == "way":
            node_refs = []
            for subelement in element.iter("nd"):
                node_refs.append(subelement.get("ref"))
            node["node_refs"] = node_refs
        
        return node
    else:
        return None

The process_map function receives the OSM input file and inserts it to the database. Due to the OSM file size for the Seattle area (1.61 GB) this function will insert each element at a time in the database to prevent jupyter notebook from reaching its memory limit.

In [33]:
def process_map(file_in, db_table):
    data = []
    i = 0
    for _, element in ET.iterparse(file_in):
        el = shape_element(element)
        if el != None:
            data.append(el)
            i = i + 1
            #Insert every 10,000 records to the database
            if i == 10000:
                db_table.insert_many(data)
                #Empty data list and restart count
                data[:] = []
                i = 0
    #Insert rest of the data list to the database
    db_table.insert_many(data)

The get_db function returns the MongoDB database instance.

In [7]:
def get_db():
    from pymongo import MongoClient
    client = MongoClient('localhost:27017')
    db = client.seattle
    return db

The code below will use the functions declared above and load the OSM file and save it to the MongoDB database

In [34]:
db = get_db()

db.seattle_data.drop()

process_map(OSMFILE, db.seattle_data)

In the query below we will count the number of documents that we imported:

In [35]:
db.seattle_data.find().count()  

7940891

Now we will count the number of nodes and ways:

In [39]:
db.seattle_data.find({"type":"node"}).count()

7233040

In [40]:
db.seattle_data.find({"type":"way"}).count()

707752

We could also investigate some other characteristics of the dataset, such as the top 10 types of parking structures:

In [41]:
match = {"$match":{"amenity":{"$eq":"parking"}}}
group = {"$group":{"_id":"$parking", "count":{"$sum":1}}}
sort = {"$sort":{"count": -1}}
limit = {"$limit" : 10}
result = db.seattle_data.aggregate([match, group, sort, limit])

for r in result:
    pprint.pprint(r)

{u'_id': None, u'count': 6037}
{u'_id': u'surface', u'count': 3522}
{u'_id': u'multi-storey', u'count': 149}
{u'_id': u'underground', u'count': 42}
{u'_id': u'rooftop', u'count': 17}
{u'_id': u'carports', u'count': 10}
{u'_id': u'garage', u'count': 6}
{u'_id': u'park_and_ride', u'count': 5}
{u'_id': u'lane', u'count': 1}
{u'_id': u'Asphalt', u'count': 1}


Or the top 10 cities with the highest number of records:

In [47]:
match = {"$match":{"address.city":{"$exists":1}}}
group = {"$group":{"_id":"$address.city", "count":{"$sum":1}}}
sort = {"$sort":{"count": -1}}
limit = {"$limit" : 10}
result = db.seattle_data.aggregate([match, group, sort, limit])

for r in result:
    pprint.pprint(r)

{u'_id': u'Seattle', u'count': 203295}
{u'_id': u'Kirkland', u'count': 42285}
{u'_id': u'Saanich', u'count': 11625}
{u'_id': u'Mount Vernon', u'count': 11524}
{u'_id': u'Langford', u'count': 2810}
{u'_id': u'Oak Bay', u'count': 2298}
{u'_id': u'Colwood', u'count': 1985}
{u'_id': u'Sooke', u'count': 1600}
{u'_id': u'Esquimalt', u'count': 1495}
{u'_id': u'View Royal', u'count': 1012}


A small portion of this dataset is located in Canada. In the next steps we are going to investigate if the records correctly reference the country in which they are located. We will start by grouping the country field and seeing which values are being used:

In [48]:
group = {"$group":{"_id":"$address.country", "count":{"$sum":1}}}
limit = {"$limit": 100}
result = db.seattle_data.aggregate([group, limit])

for r in result:
    pprint.pprint(r)

{u'_id': u'US', u'count': 2466}
{u'_id': u'CA', u'count': 50}
{u'_id': None, u'count': 7938375}


The query above shows that the majority of the records do not include country information. As an alternative we could investigate records which have a province or state record:

In [49]:
group = {"$group":{"_id":"$address.province", "count":{"$sum":1}}}
limit = {"$limit": 100}
result = db.seattle_data.aggregate([group, limit])

for r in result:
    pprint.pprint(r)

{u'_id': u'British Columbia', u'count': 5}
{u'_id': u'BC', u'count': 136}
{u'_id': None, u'count': 7940750}


The query results above shows that there are a few records without a country which are located in British Columbia. We will set the country as Canada for those records an standardize all of them to use the same convention for the province name. After that we will set all of those records to have 'CA' as the country.

In [50]:
db.seattle_data.update_many({"address.province" : {"$eq" : "British Columbia"}}, {"$set" : {"address.province" : "BC" } })
db.seattle_data.update_many({"address.province" : {"$eq" : "BC"}}, {"$set" : {"address.country" : "CA" } })

Now we will verify how the values for the state record. The code below shows us that several fields have street names on their state field which indicates incorrect data entry.

In [51]:
group = {"$group":{"_id":"$address.state", "count":{"$sum":1}}}
limit = {"$limit": 100}
result = db.seattle_data.aggregate([group, limit])

for r in result:
    pprint.pprint(r)

{u'_id': u'WA - Washington', u'count': 1}
{u'_id': u'NE 18th Street', u'count': 1}
{u'_id': u'NE 15th Street', u'count': 1}
{u'_id': u'W', u'count': 2}
{u'_id': u'98107', u'count': 1}
{u'_id': u'Washington', u'count': 26}
{u'_id': u'wa', u'count': 53}
{u'_id': u'Wa', u'count': 5}
{u'_id': None, u'count': 7935530}
{u'_id': u'156th Avenue NE', u'count': 1}
{u'_id': u'w', u'count': 1}
{u'_id': u'BC', u'count': 1}
{u'_id': u'washington', u'count': 1}
{u'_id': u'WA.', u'count': 1}
{u'_id': u'WA', u'count': 5266}


We will update the records which has Washington, washington or wa the state to use the WA value. After that, we will update the country in the records located in Washington State.

In [54]:
db.seattle_data.update_many({"$or" : [{"address.state" : {"$eq" : "washington"}},{"address.state" : {"$eq" : "Washington"}}\
          ,{"address.state" : {"$eq" : "wa"}}, {"address.state" : {"$eq" : "Wa"}} \
          ,{"address.state" : {"$eq" : "WA."}}, {"address.state" : {"$eq" : "WA - Washington"}}\
                                     ]}, {"$set" : {"address.state" : "WA" } })
db.seattle_data.update_many({"address.state" : {"$eq" : "WA"}}, {"$set" : {"address.country" : "US" } })

It is important that a dataset such as this has complete and standardized information. This would be useful in case we were interested in grouping the data by country, state, province or another address attribute. However, this is not true for this data.

The country updates that we performed will fix the country, state and province for some of the nones. As can be verified in the query below, the number of US and CA records has increased. However, there are still some records with incorrect state and country information that we have not treated. Also, the great majority of the records do not have state, country or province information, making it more difficult to identify which country the belong to.

A more efficient way to update the adress attibutes would be to use geographic coordinates of those nodes and ways. This would be better than using the approach that we used in this project as there could be records which are incorrectly labeled as being in a particular country, but with the geographic coordinates for another country.

In [56]:
group = {"$group":{"_id":"$address.country", "count":{"$sum":1}}}
limit = {"$limit": 100}
result = db.seattle_data.aggregate([group, limit])

for r in result:
    pprint.pprint(r)

{u'_id': u'CA', u'count': 186}
{u'_id': u'US', u'count': 6124}
{u'_id': None, u'count': 7934581}
